<h1 align="center">Symbol Recognition of O/X Dataset Using MLP</h1>
<table>
    <tr><td><img src="images/o_all.png" width="300"></td><td><img src="images/x_all.png" width="300"></td></tr>
</table>

<hr>
<h3>載入資料集，驗證</h3>

In [1]:
# 載入資料集，驗證

import pickle
import cv2

with open('data/ox_train_images.pkl', 'rb') as fp:
    ox_train_images = pickle.load(fp)
fp.close()

with open('data/ox_train_labels.pkl', 'rb') as fp:
    ox_train_labels = pickle.load(fp)
fp.close()

with open('data/ox_test_images.pkl', 'rb') as fp:
    ox_test_images = pickle.load(fp)
fp.close()

with open('data/ox_test_labels.pkl', 'rb') as fp:
    ox_test_labels = pickle.load(fp)
fp.close()

cv2.namedWindow('O/X')
n = len(ox_train_images)
for i in range(n):
    z = ox_train_images[i,:,:]
    cv2.imshow('O/X', z)
    cv2.waitKey(100)
n = len(ox_test_images)
for i in range(n):
    z = ox_test_images[i,:,:]
    cv2.imshow('O/X', z)
    cv2.waitKey(100)
cv2.destroyAllWindows()


<hr>
<h3>觀察樣本資料維度</h3>

In [2]:
# 觀察樣本資料維度

print(ox_train_images.shape)
print(ox_train_labels.shape)
print(ox_test_images.shape)
print(ox_test_labels.shape)


(80, 32, 32)
(80, 2)
(20, 32, 32)
(20, 2)


<hr>
<h3>扁平化（Flatten）訓練影像（二維影像轉換為一維特徵向量）</h3>

In [3]:
# 扁平化（Flatten）訓練影像（二維影像轉換為一維特徵向量）

x0 = ox_train_images.reshape(80,32*32)
y0 = ox_train_labels

print(x0.shape)
print(y0.shape)


(80, 1024)
(80, 2)


<hr>
<h3>建立訓練集 Tensors</h3>

In [4]:
# 建立訓練集 Tensors

import torch

x = torch.from_numpy(x0).float()
y = torch.from_numpy(y0).float()

print(x)
print(y)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        

<hr>
<h3>建立 MLP 模型</h3>

In [5]:
# 建立 MLP 模型

import torch.nn as nn

class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        # 1024 inputs -> 128 hiddens --> 16 hiddens --> 2 outputs
        self.fc1 = nn.Linear(1024, 128)
        self.fc2 = nn.Linear(128, 16)
        self.fc3 = nn.Linear(16, 2)
    def forward(self, x):
        x = self.fc1(x).sigmoid()
        x = self.fc2(x).sigmoid()
        x = self.fc3(x).sigmoid()
        return x

mlp = MLPNet()

print(mlp)


MLPNet(
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=2, bias=True)
)


<hr>
<h3>MLP 訓練</h3>

In [6]:
# MLP 訓練

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 樣本數
N = len(x)

EPOCH = 1000

# preparing training samples
# (Ready)

# shuffling definition
def shuffling(x, y):
    x_out = torch.randn(N, 1024)
    y_out = torch.randn(N, 2)
    idx = [i for i in range(N)]
    idx = np.random.permutation(idx)
    for i in range(N):
        x_out[i] = x[idx[i]]
        y_out[i] = y[idx[i]]
    return x_out, y_out

# Loss function
criterion = nn.MSELoss()
# Optimizer
# optimizer = optim.SGD(mlp.parameters(), lr=0.01)
optimizer = optim.Adam(mlp.parameters(), lr=0.001)

# training
for epoch in range(EPOCH):
    # zero the gradient buffers
    optimizer.zero_grad()
    # shuffling
    x_new, y_new = shuffling(x, y)
    # feed foreward
    output = mlp(x_new)
    # evaluating loss
    loss = criterion(output, y_new)
    # display loss
    if (epoch % 10 == 0):
        print('epoch = %6d, loss = %12.8f' % (epoch, loss))
    # feed backward
    loss.backward()
    # update parameters
    optimizer.step()


epoch =      0, loss =   0.25138551
epoch =     10, loss =   0.17715217
epoch =     20, loss =   0.14235599
epoch =     30, loss =   0.11685989
epoch =     40, loss =   0.09833296
epoch =     50, loss =   0.08440344
epoch =     60, loss =   0.07349205
epoch =     70, loss =   0.06463897
epoch =     80, loss =   0.05718330
epoch =     90, loss =   0.05078288
epoch =    100, loss =   0.04528184
epoch =    110, loss =   0.04059545
epoch =    120, loss =   0.03661466
epoch =    130, loss =   0.03321796
epoch =    140, loss =   0.03029745
epoch =    150, loss =   0.02776594
epoch =    160, loss =   0.02555485
epoch =    170, loss =   0.02361027
epoch =    180, loss =   0.02188951
epoch =    190, loss =   0.02035838
epoch =    200, loss =   0.01898919
epoch =    210, loss =   0.01775860
epoch =    220, loss =   0.01664917
epoch =    230, loss =   0.01564481
epoch =    240, loss =   0.01473226
epoch =    250, loss =   0.01390043
epoch =    260, loss =   0.01313987
epoch =    270, loss =   0.0

<hr>
<h3>扁平化（Flatten）測試影像（二維影像轉換為一維特徵向量）</h3>

In [7]:
# 扁平化（Flatten）測試影像（二維影像轉換為一維特徵向量）

x0 = ox_test_images.reshape(20,32*32)
y0 = ox_test_labels

print(x0.shape)
print(y0.shape)


(20, 1024)
(20, 2)


<hr>
<h3>建立測試集 Tensors</h3>

In [8]:
# 建立測試集 Tensors

import torch

xt = torch.from_numpy(x0).float()
yt = torch.from_numpy(y0).float()

print(xt)
print(yt)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])


<hr>
<h3>饋入 MLP 整批測試</h3>

In [9]:
# 饋入 MLP 整批測試

output = mlp(xt)

olst = output.tolist()

n = len(olst)

correct = 0
for i in range(n):
    o = olst[i]
    if (o[0] > o[1]) and (yt[i][0] > yt[i][1]):
        correct = correct + 1
    if (o[0] < o[1]) and (yt[i][0] < yt[i][1]):
        correct = correct + 1
    t = '[%8.6f,%8.6f] vs. [%d,%d]' % (o[0], o[1], yt[i][0], yt[i][1])
    print(t)

print('Accuracy = %d / %d' % (correct, n))


[0.966418,0.043954] vs. [1,0]
[0.966418,0.043954] vs. [1,0]
[0.966418,0.043954] vs. [1,0]
[0.966392,0.043989] vs. [1,0]
[0.966418,0.043954] vs. [1,0]
[0.966418,0.043954] vs. [1,0]
[0.966418,0.043954] vs. [1,0]
[0.966300,0.044102] vs. [1,0]
[0.966315,0.044081] vs. [1,0]
[0.966418,0.043954] vs. [1,0]
[0.035467,0.956197] vs. [0,1]
[0.036231,0.955254] vs. [0,1]
[0.035404,0.956278] vs. [0,1]
[0.035404,0.956278] vs. [0,1]
[0.035444,0.956225] vs. [0,1]
[0.035515,0.956135] vs. [0,1]
[0.036200,0.955288] vs. [0,1]
[0.037937,0.953233] vs. [0,1]
[0.035404,0.956278] vs. [0,1]
[0.035648,0.955970] vs. [0,1]
Accuracy = 20 / 20
